In [202]:
import warnings
warnings.filterwarnings('ignore')

In [203]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [204]:
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, classification_report
from imblearn.ensemble import EasyEnsembleClassifier




# Read the CSV and Perform Basic Data Cleaning

In [205]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [206]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [207]:
df.select_dtypes(include=[object]).tail()

,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
68812,RENT,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68813,RENT,Not Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68814,MORTGAGE,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68815,MORTGAGE,Verified,Jan-2019,low_risk,n,f,May-2019,Individual,N,N
68816,MORTGAGE,Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N


In [208]:
df_encoded = pd.get_dummies(df, columns=["home_ownership", "pymnt_plan", "verification_status", "issue_d", "next_pymnt_d", "initial_list_status", "application_type", "hardship_flag", "debt_settlement_flag"]).copy()
df_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Jan-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,1,0,1,0,1,1,0,1,1


# Split the Data into Training and Testing

In [209]:
# Create our features
X = df_encoded.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [210]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Jan-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.451066,0.177238,0.383161,0.616839,0.123879,0.876121,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.497603,0.381873,0.486161,0.486161,0.329446,0.329446,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [211]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [212]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

In [213]:
Counter(y_train)

Counter({'low_risk': 51366, 'high_risk': 246})

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [214]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model = rf_model.fit(X_train, y_train)
rf_model

BalancedRandomForestClassifier(bootstrap=True, class_weight=None,
                               criterion='gini', max_depth=None,
                               max_features='auto', max_leaf_nodes=None,
                               min_impurity_decrease=0.0, min_samples_leaf=2,
                               min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=100,
                               n_jobs=1, oob_score=False, random_state=1,
                               replacement=False, sampling_strategy='auto',
                               verbose=0, warm_start=False)

In [215]:
# Calculated the balanced accuracy score
predictions = rf_model.predict(X_test)
balanced_score_rf = round(balanced_accuracy_score(y_test, predictions),2)*100
print(f"Balanced Random Forest Classifier Accuracy Score: {balanced_score_rf}%")

Balanced Random Forest Classifier Accuracy Score: 78.0%


In [216]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[   67,    34],
       [ 1936, 15168]], dtype=int64)

In [217]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
results = classification_report_imbalanced(y_test, predictions)
print(f"Balanced Random Forest, Classification Report")
print(results)

Balanced Random Forest, Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.66      0.89      0.06      0.77      0.58       101
   low_risk       1.00      0.89      0.66      0.94      0.77      0.60     17104

avg / total       0.99      0.89      0.66      0.93      0.77      0.60     17205



In [218]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

for i, j in importances_sorted:
    print (j, i)

total_rec_prncp 0.0749454618963767
total_pymnt_inv 0.06890205772422582
total_rec_int 0.0642230472654842
last_pymnt_amnt 0.05312874913917733
total_pymnt 0.05124404056219496
int_rate 0.03433254605108269
mths_since_recent_inq 0.018430627002275348
issue_d_Jan-2019 0.018233715594670658
installment 0.01659302999836689
total_bal_il 0.016420565463631808
revol_bal 0.016148867709336906
mths_since_rcnt_il 0.016054301958682963
out_prncp_inv 0.01596736493075527
mo_sin_old_rev_tl_op 0.015744769753312588
total_rev_hi_lim 0.015695977587892995
tot_cur_bal 0.01565407348654215
out_prncp 0.015495376369280169
mo_sin_old_il_acct 0.015315643426436248
tot_hi_cred_lim 0.015077508375261686
dti 0.014963509037529497
annual_inc 0.014566336472184997
max_bal_bc 0.014190282464106412
issue_d_Mar-2019 0.013976831742311971
avg_cur_bal 0.013657370277281361
all_util 0.013546710541006937
total_bc_limit 0.013228657344512207
loan_amnt 0.01304287158190207
bc_util 0.012748384194412656
total_bal_ex_mort 0.012517439184641039
bc_

### Easy Ensemble AdaBoost Classifier

In [219]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

ensemble_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ensemble_model = ensemble_model.fit(X_train, y_train)
ensemble_model


EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=1,
                       random_state=1, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [220]:
# Calculated the balanced accuracy score
predictions = ensemble_model.predict(X_test)

balanced_score_en = round(balanced_accuracy_score(y_test, predictions),2)*100
print(f"Easy Ensemble Classifier Accuracy Score: {balanced_score_en}%")

Easy Ensemble Classifier Accuracy Score: 93.0%


In [221]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[   93,     8],
       [  977, 16127]], dtype=int64)

In [222]:
# Print the imbalanced classification report
results = (classification_report_imbalanced(y_test, predictions))
print("Easy Ensemble Classifier, Classification Report")
print(results)

Easy Ensemble Classifier, Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



In [223]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(random_state=1)
model.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=1)

In [224]:
predictions = model.predict(X_test)
balanced_score_ad = round(balanced_accuracy_score(y_test, predictions),2)*100
print(f"AdaBoost Classifier Accuracy Score: {balanced_score_ad}%")

AdaBoost Classifier Accuracy Score: 69.0%


In [225]:
print(confusion_matrix(y_test, predictions))

[[   39    62]
 [    4 17100]]


In [226]:
results = (classification_report_imbalanced(y_test, predictions))
print("AdaBoost Classifier, Classification Report")
print(results)


AdaBoost Classifier, Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.91      0.39      1.00      0.54      0.62      0.36       101
   low_risk       1.00      1.00      0.39      1.00      0.62      0.41     17104

avg / total       1.00      1.00      0.39      1.00      0.62      0.41     17205

